In [0]:
import numpy as np # For QTable
import gym  # OpenAI Gym for FrozenLake implementation
import random # Generate random numbers

from tqdm.notebook import tqdm # Show progress while training

## Setting up environment 🏗️

### Creating the environment 🎮

In [0]:
env = gym.make("Taxi-v3")

### Create Q-Table and initialize 🗄️


The Q-table will need to be a (state_size X action_size) table.  

OpenAI Gym provides this information in the form of variable

In [0]:
action_size = env.action_space.n
state_size = env.observation_space.n

print("State Size:", state_size, " Action Size:", action_size)

State Size: 500  Action Size: 6


In [0]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## Hyperparamenters ⚙️

In [0]:
total_episodes = 40000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 150               # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005           # Exponential decay rate for exploration prob

## Q-Learning 🧠

In [0]:
# List of rewards
rewards = []


# 2 For life or until learning is stopped
for episode in tqdm(range(total_episodes)):

    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    # Limiting the search space to max_steps
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
          
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)


Score over time: 5.29975
[[  0.           0.           0.           0.           0.
    0.        ]
 [  2.73726806   3.94935372   2.7392535    3.94935536   5.20997639
   -5.11252871]
 [  7.93346352   9.37000425   7.9333463    9.40364088  10.9512375
    0.40277854]
 ...
 [ 10.95123745  12.58025     10.95037738   9.40352731   1.94995613
    1.92942015]
 [ -5.17223369   6.53316196  -5.18737206  -5.22091109 -12.70899548
  -12.7099007 ]
 [ 12.69423201  13.97409921  14.64917361  18.           4.07999175
    6.79023792]]


## Use our Q-Table to play Taxi 🚕

In [0]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Number of steps 15
****************************************************
EPISODE  1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Number of steps 12
****************************************************
EPISODE  2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Number of steps 10
****************************************************
EPISODE  3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Number of steps 8
****************************************************
EPISODE  4
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Number of steps 12


## Visualize a complete game 👨‍💻

In [0]:
state = env.reset()
step = 0
done = False

for step in range(max_steps):
    env.render()

    # Take the action (index) that have the maximum expected future reward given that state
    action = np.argmax(qtable[state,:])
    
    new_state, reward, done, info = env.step(action)
    
    if done: 
        env.render()       
        # We print the number of step it took.
        print("Number of steps", step)
        break
    state = new_state

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| :_| : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |